## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Provideniya,RU,64.3833,-173.3000,5.76,84,99,3.40,overcast clouds
1,1,Tasiilaq,GL,65.6145,-37.6368,14.00,44,20,2.30,few clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,53.60,62,75,8.32,light rain
3,3,New Norfolk,AU,-42.7826,147.0587,57.00,69,54,2.42,broken clouds
4,4,Dikson,RU,73.5069,80.5464,-18.47,90,96,4.63,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you want?'))
max_temp = float(input('What is the maximum temperature you want?'))

What is the minimum temperature you want?60
What is the maximum temperature you want?85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)].dropna()
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Carupano,VE,10.6678,-63.2585,80.96,73,77,14.65,light rain
9,9,Hilo,US,19.7297,-155.0900,69.80,78,40,9.22,scattered clouds
12,12,Saint-Philippe,RE,-21.3585,55.7679,82.40,74,47,2.30,scattered clouds
13,13,Georgetown,MY,5.4112,100.3354,80.01,89,20,1.30,few clouds
17,17,Cap Malheureux,MU,-19.9842,57.6142,79.00,76,8,1.99,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                323
City                   323
Country                323
Lat                    323
Lng                    323
Max Temp               323
Humidity               323
Cloudiness             323
Wind Speed             323
Current Description    323
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Carupano,VE,10.6678,-63.2585,80.96,73,77,14.65,light rain
9,9,Hilo,US,19.7297,-155.0900,69.80,78,40,9.22,scattered clouds
12,12,Saint-Philippe,RE,-21.3585,55.7679,82.40,74,47,2.30,scattered clouds
13,13,Georgetown,MY,5.4112,100.3354,80.01,89,20,1.30,few clouds
17,17,Cap Malheureux,MU,-19.9842,57.6142,79.00,76,8,1.99,clear sky


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Carupano,VE,80.96,light rain,10.6678,-63.2585,
9,Hilo,US,69.80,scattered clouds,19.7297,-155.0900,
12,Saint-Philippe,RE,82.40,scattered clouds,-21.3585,55.7679,
13,Georgetown,MY,80.01,few clouds,5.4112,100.3354,
17,Cap Malheureux,MU,79.00,clear sky,-19.9842,57.6142,
19,Kapaa,US,73.40,light rain,22.0752,-159.3190,
20,Itigi,TZ,64.06,overcast clouds,-5.7000,34.4833,
23,San Policarpo,PH,78.93,overcast clouds,12.1791,125.5072,
26,Bredasdorp,ZA,66.20,few clouds,-34.5322,20.0403,
29,Ribeira Grande,PT,62.60,light rain,38.5167,-28.7000,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Carupano,VE,80.96,light rain,10.6678,-63.2585,Hotel La dorada
9,Hilo,US,69.80,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
12,Saint-Philippe,RE,82.40,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
13,Georgetown,MY,80.01,few clouds,5.4112,100.3354,Cititel Penang
17,Cap Malheureux,MU,79.00,clear sky,-19.9842,57.6142,Pereybere Hotel & Spa


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weath Descr</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))